In [2]:
import pywt

import tensorflow as tf
from mit_bih_data_maker import mit_bih_data_maker

def test_label_maker(label):
    label_map = {"N":0, "L":1, "R":2, "/":3, "V":4, "A":5}
    if label in label_map:
        return label_map[label]
    return "Omit"

data_maker = mit_bih_data_maker()
data_maker.config("../database/mit-bih/", "../database/mit-bih", "../database/mit-bih-r-label")
#rec_list = ["100", "109", "118", "107", "208", "232"]
rec_list = []
data_maker.load_data(rec_list)

sample_num = -1
label_percent = {0:0.197, 1:0.220, 2:0.191, 3:0.183, 4:0.087, 5:0.122}
train_percent = 0.7
ecg_data = data_maker.make_by_r_peak(before_r = 90, after_r = 146, sample_num = sample_num, label_maker = test_label_maker, \
                                     label_percent = label_percent, train_percent = train_percent)
ecg_data.print_summary()

TRAIN Summay
Sample num: 14473, feature num: 236
Sample num for each label:
Label 0: 2879
Label 1: 3167
Label 2: 2783
Label 3: 2628
Label 4: 1234
Label 5: 1782

TEST Summary
Sample num: 6203, feature num: 236
Sample num for each label:
Label 0: 1194
Label 1: 1382
Label 2: 1166
Label 3: 1156
Label 4: 565
Label 5: 740


In [19]:
import numpy as np
from statsmodels.tsa.ar_model import AR

db8 = pywt.Wavelet('db8')
fea_array = []
for i in xrange(ecg_data.train.x.shape[0]):
    cA3, cD3, cD2, cD1 = pywt.wavedec(ecg_data.train.x[i].tolist(), db8, level = 3)
    model = AR(ecg_data.train.x[i].tolist())
    result = model.fit(maxlag = 4)
    fea_array.append(cA3.tolist() + result.params[1:].tolist())
    
print np.array(fea_array).shape

(14473, 46)


In [20]:
from sklearn import svm

clf = svm.SVC(decision_function_shape='ovo')
clf.fit(np.array(fea_array), ecg_data.train.y)
print clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [21]:
from sklearn.metrics import accuracy_score

fea_array = []
for i in xrange(ecg_data.test.x.shape[0]):
    cA3, cD3, cD2, cD1 = pywt.wavedec(ecg_data.test.x[i].tolist(), db8, level = 3)
    model = AR(ecg_data.test.x[i].tolist())
    result = model.fit(maxlag = 4)
    fea_array.append(cA3.tolist() + result.params[1:].tolist())
    wt_array.append(cA3)
    
y_pred = clf.predict(np.array(fea_array))
print accuracy_score(y_pred, ecg_data.test.y)

0.95421570208
